# __영화 평점 데이터를 사용하여 영화 추천 모델 만들기__

**추천 모델 이해하기** 
--- 
👉 일반적으로 추천 모델에서 해당 플랫폼의 특정 사용자는 플랫폼 내의 전체 아이템(제품 또는 콘텐츠)이 아닌 소수의 아이템만을 소비(시청여부, 구매이력, 후기, 평점 등)하게 됩니다. 아직 소비하지 않은 아이템에 대해서 특정 소비자가 어떤 평점을 줄지 예측하고 높은 평점을 줄 것으로 예측되는 아이템(하지만 소비되지 않은 아이템)을 예측 평점이 높은 순위부터 보여줍니다.


**본 튜토리얼에서는**
--- 
👉 사용자가 콘텐츠에 부여한 평점 데이터를 사용하여 간단하게 사용자가 좋아할만한 영화 목록을 추천해주는 추천시스템을 구축합니다. 이를 위해, Movielens 영화 평점 데이터 세트를 사용합니다. Movielens 데이터 세트는 사용자가 특정 영화에 대해서 평점을 준 것을 모아놓은 데이터 세트입니다.

## __0. 데이터 세트 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/quick_start/how_to_use_ThanoSQL/#5-thanosql)
에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.   

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN={발급받은 개인 토큰}

In [16]:
%%thanosql
COPY movielens_data FROM "tutorial_data/movielens_data/movielens.csv"

Success


## __1. 데이터 세트 확인__

영화 평점 데이터를 이용한 영화 추천 모델을 만들기 위해 ThanoSQL DB에 저장되어 있는 movielens_data 테이블을 사용합니다. 아래의 쿼리문을 실행하고 테이블의 내용을 확인합니다.

In [17]:
%%thanosql 
SELECT * 
FROM movielens_data 
LIMIT 5

,userid,movieid,rating,title
0,31,1,3.0,Toy Story (1995)
1,31,110,5.0,Braveheart (1995)
2,31,260,5.0,Star Wars: Episode IV - A New Hope (1977)
3,31,364,3.0,"Lion King, The (1994)"
4,31,527,0.5,Schindler's List (1993)


__데이터 이해하기__ 

movielens_data 데이터 세트는 다음과 같은 정보를 담고 있습니다.  
- userid : 사용자 ID   
- movieid : 영화 ID  
- rating : 평점   
- title : 영화 제목  

userid 값이 31인 사용자는 "Toy Story (1995)"에 평점 3.0점을 주고 "Braveheart (1995)"에 평점 5.0점을 준 것을 확인할 수 있습니다. 

## __2. 추천 모델 생성__

이전 단계에서 확인한 movielens_data 데이터를 사용하여 영화 추천 모델을 만듭니다. 다음 쿼리 구문을 실행하여 my_movie_rec 이름의 모델을 만듭니다.

(쿼리 실행 시 예상 소요 시간 : 1 min)


In [20]:
%%thanosql
BUILD MODEL my_movie_rec
USING Light_FM
OPTIONS (
    user_col='userid',   
    item_col='movieid',
    rating_col='rating',
    description_col='title'
    )
AS 
SELECT * 
FROM movielens_data

Success


__쿼리 세부 정보__ 
- "__BUILD MODEL__" 쿼리 구문을 사용하여 <mark style="background-color:#E9D7FD ">my_movie_rec</mark> 이라는 모델을 만들고 학습시킵니다. <br>
- "__OPTIONS__"에서는 3개의 컬럼(Column)이 사용합니다. "user_col"에는 사용자의 ID (<mark style="background-color:#D7D0FF ">userid</mark>), "item_col"에는 아이템의 ID가 들어갑니다. 본 튜토리얼에서는 <mark style="background-color:#D7D0FF ">movieid</mark>가 사용됩니다. "rating_col"에서는 예측하고자 하는 목표값이 되는 열의 이름(<mark style="background-color:#D7D0FF ">rating</mark>)을 적어줍니다. 

## __3. 생성된 모델을 사용하여 추천 목록 생성__

이전 단계에서 생성한 영화 추천 모델을 사용해서 특정 사용자가 좋아할만한 아이템(영화) 목록을 10개 생성합니다.

In [22]:
%%thanosql
PREDICT USING my_movie_rec
OPTIONS (
    predict_type='user',
    user_id=31, 
    nrec=10
    )
AS 
SELECT * 
FROM movielens_data

,rank
0,"Incredibles, The (2004)"
1,Avatar (2009)
2,"Avengers, The (2012)"
3,Toy Story 3 (2010)
4,Iron Man (2008)
5,Harry Potter and the Half-Blood Prince (2009)
6,How to Train Your Dragon (2010)
7,Night at the Museum (2006)
8,Up (2009)
9,Finding Nemo (2003)


__쿼리 세부 정보__ 
- __"PREDICT USING"__ 쿼리 구문을 사용하여 이전 단계에서 만든 my_movie_rec 모델을 예측에 사용합니다. 
- 추천 모델에서는 예측 단계에서도 "__OPTIONS__"를 사용합니다. "predict_type"은 예측 결과를 정렬할 대상 기준을 설정합니다. 이번 튜토리얼에서는 특정 사용자(userid의 값이 31)에게 추천할 영화 목록을 보려고 하기 때문에 "itemid"를 적어주며 "itemid"는 보고자 하는 특정 사용자의 userid값인 31을 입력합니다. "nrec"는 추천하는 아이템의 개수를 의미합니다. 